<a href="https://colab.research.google.com/github/luanfelixcoding/climate-change-impact-economy/blob/master/climate_change_impact_economy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Link para base de dados:** https://www.kaggle.com/datasets/tarunrm09/climate-change-indicators

**Link para base de dados:** https://www.kaggle.com/datasets/sazidthe1/global-inflation-data?resource=download

In [ ]:
# conectando com o drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
climate_change_file_path: str = "/content/drive/My Drive/datasets/climate_change_dataset/climate_change_indicators.csv"
global_inflation_file_path : str = "/content/drive/My Drive/datasets/global_inflation_dataset/global_inflation_data.csv"

# datasets
df_climate = pd.read_csv(climate_change_file_path)
df_economy = pd.read_csv(global_inflation_file_path)

In [ ]:
df_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             225 non-null    int64  
 1   Country              225 non-null    object 
 2   ISO2                 223 non-null    object 
 3   ISO3                 225 non-null    object 
 4   Indicator            225 non-null    object 
 5   Unit                 225 non-null    object 
 6   Source               225 non-null    object 
 7   CTS_Code             225 non-null    object 
 8   CTS_Name             225 non-null    object 
 9   CTS_Full_Descriptor  225 non-null    object 
 10  F1961                188 non-null    float64
 11  F1962                189 non-null    float64
 12  F1963                188 non-null    float64
 13  F1964                188 non-null    float64
 14  F1965                188 non-null    float64
 15  F1966                192 non-null    flo

In [ ]:
df_economy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    196 non-null    object 
 1   indicator_name  196 non-null    object 
 2   1980            140 non-null    float64
 3   1981            144 non-null    float64
 4   1982            145 non-null    float64
 5   1983            145 non-null    float64
 6   1984            145 non-null    float64
 7   1985            145 non-null    float64
 8   1986            145 non-null    float64
 9   1987            147 non-null    float64
 10  1988            147 non-null    float64
 11  1989            147 non-null    float64
 12  1990            150 non-null    float64
 13  1991            155 non-null    float64
 14  1992            158 non-null    float64
 15  1993            169 non-null    float64
 16  1994            171 non-null    float64
 17  1995            172 non-null    flo

In [ ]:
# verificando valores nulos em cada coluna
def checking_null_values(df: pd.DataFrame) -> None:
  for col in df.columns:
    if df[col].isnull().sum() > 0:
      print(f"{df[col].name} = {df[col].isnull().sum()}")

In [ ]:
# Pegando apenas as colunas que representam anos
years = [col for col in df_climate.columns if col.startswith('F')]

# Derrete o dataset em variaveis e valores
# 'Country' e 'ISO3' mantemos, nomeamos uma coluna como 'Year' que recebe a variavel 'years'
df_climate_melted = df_climate.melt(
    id_vars=['Country', 'ISO3'],
    value_vars=years,
    var_name='Year',
    value_name='Temperature_Change'
)
df_climate_melted.rename(columns={'Country' : 'Country_Name'}, inplace=True)

# Na coluna 'Year' extraindo todos os numeros e convertendo para int, "(\d+)" permite extrair numeros compostos
df_climate_melted['Year'] = df_climate_melted['Year'].str.extract(r"(\d+)").astype(int)
# checking_null_values(df_climate_melted) # output: Temperature_Change = 1490

# Removendo valores nulos; inplace=True faz as alterações no proprio df, inplace=False cria outro df com os valores removidos
df_climate_melted.dropna(subset=['Temperature_Change'], inplace=True)
checking_null_values(df_climate_melted) # output: Temperature_Change = 0
df_climate_melted.head()

,Country_Name,ISO3,Year,Temperature_Change
0,"Afghanistan, Islamic Rep. of",AFG,1961,-0.113
1,Albania,ALB,1961,0.627
2,Algeria,DZA,1961,0.164
3,American Samoa,ASM,1961,0.079
4,"Andorra, Principality of",AND,1961,0.736


## Plotando Gráfico referente as mudanças climáticas durante os anos

In [ ]:
# Agrupando a coluna 'Year' e pegando a media de seus valores em conjunto
global_temp_trend = df_climate_melted.groupby('Year', as_index=False)['Temperature_Change'].mean().sort_values(by='Year')

# Criando a visualização
fig_global = go.Figure()

fig_global.add_trace(go.Scatter(
    x=global_temp_trend['Year'],
    y=global_temp_trend['Temperature_Change'],
    mode='lines+markers',
    line=dict(color='red', width=2),  # Linha vermelha
    marker=dict(color='black', size=5),  # Pontos pretos
    name='Tendência Global'
))


fig_global.update_layout(
    title="Tendência Global da Mudança de Temperatura (1961-2022)",
    xaxis_title="Ano",
    yaxis_title="Mudança Média de Temperatura (°C)",
    template="plotly_white",
    showlegend=True
)

# Exibir o gráfico
fig_global.show()

In [ ]:
# Calculando a variação de temperatura por país (máximo - mínimo)
df_climate_variation = df_climate_melted.groupby("Country_Name")["Temperature_Change"].agg(lambda x: x.max() - x.min())

# Selecionando os 10 países mais afetados (com maior variação)
top_10_countries = df_climate_variation.nlargest(10).index
df_top_10_countries = df_climate_melted[df_climate_melted['Country_Name'].isin(top_10_countries)].sort_values(by=['Year'])

# Criando o gráfico interativo
fig_countries = px.line(df_top_10_countries, x="Year", y="Temperature_Change", color="Country_Name",
                        title="Comparação dos Países com Maior Variação ao longo dos Anos",
                        labels={"Temperature_Change": "Mudança Média de Temperatura (°C)", "Year": "Ano"},
                        markers=True)

fig_countries.update_traces(line=dict(width=2))
fig_countries.show()


In [ ]:
df_economy_melted = df_economy.melt(
    id_vars=['country_name'],
    value_vars=[str(col) for col in range(1980, 2025)],
    var_name='Year',
    value_name='Inflation_Rate'
)

df_economy_melted['Year'] = df_economy_melted['Year'].astype(int)
df_economy_melted.head()

,country_name,Year,Inflation_Rate
0,Afghanistan,1980,13.4
1,Albania,1980,NaN
2,Algeria,1980,9.7
3,Andorra,1980,NaN
4,Angola,1980,46.7
